# **CIS 520: Machine Learning, Fall 2022**
# **Week 11, Worksheet 3**
## **Hidden Markov Models**


- **Content Creator:** Yide Zhao, Jasleen Dhanoa
- **Content Checkers:** Gautam Ramesh, Yang Yan

**Hidden Markov Models (HMM)**

---



## **Autograding and the PennGrader**

First, you'll need to set up the PennGrader, which we'll be using throughout the semester to help you with your homeworks and worksheeets.

PennGrader is not only **awesome**, but it was built by an equally awesome person: Leo Murri.  Today, Leo works as a data scientist at Amazon!

PennGrader was developed to provide students with *instant* feedback on their answer. You can submit your answer and know whether it's right or wrong instantly. We then record your most recent answer in our backend database.

### Imports and Setup (Do Not Modify This Section)

In [1]:
%%capture
!pip install penngrader


In [2]:
import random 
import numpy as np
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
from numpy.linalg import *
np.random.seed(42)  # don't change this line

import dill
import base64

In [3]:
# For autograder only, do not modify this cell. 
# True for Google Colab, False for autograder
NOTEBOOK = (os.getenv('IS_AUTOGRADER') is None)
if NOTEBOOK:
    print("[INFO, OK] Google Colab.")
else:
    print("[INFO, OK] Autograder.")
    sys.exit()

[INFO, OK] Google Colab.


### Insert PennID here!

In [4]:
#PLEASE ENSURE YOUR PENN-ID IS ENTERED CORRECTLY. IF NOT, THE AUTOGRADER WON'T KNOW WHO 
#TO ASSIGN POINTS TO YOU IN OUR BACKEND
STUDENT_ID = 57931095 # YOUR PENN-ID GOES HERE AS AN INTEGER#

In [5]:
import penngrader.grader

grader = penngrader.grader.PennGrader(homework_id = 'CIS_5200_202230_HW_HMM_WS', student_id = STUDENT_ID)

PennGrader initialized with Student ID: 57931095

Make sure this correct or we will not be able to store your grade


In [6]:
# A helper function for grading utils
def grader_serialize(obj):        # A helper function
    '''Dill serializes Python object into a UTF-8 string'''
    byte_serialized = dill.dumps(obj, recurse = True)
    return base64.b64encode(byte_serialized).decode("utf-8")

Let's work through an example of an HMM to see how probability propagates and to find the hidden states.


Example: On any given day, Alice is in one of two states: happy or sad. You do not know her internal state, but get to observe her activities in the evening. Each evening, she either sings, goes for a walk, or watches TV.

Alice's state on any day is random. Her state $Z_{1}$ on day 1 is equally likely to be happy or sad:
$$
P\left(Z_{1}=\text { happy }\right)=1 / 2
$$
Given her state $Z_{t}$ on day $t,$ her state $Z_{t+1}$ on the next day is governed by the following probabilities (and is Markovian: conditionally independent of her previous states and activities):
$$
P\left(Z_{t+1}=\text { happy } \mid Z_{t}=\text { happy }\right)=4 / 5 \quad P\left(Z_{t+1}=\text { happy } \mid Z_{t}=\mathrm{sad}\right)=1 / 2
$$
Alice's activities are also random. Her activities vary based on her state; given her state $Z_{t}$ on day $t,$ her activity $X_{t}$ on that day is governed by the following probabilities (and is conditionally independent of everything else $)$
$$
\begin{array}{ll}
P\left(X_{t}=\operatorname{sing} \mid Z_{t}=\text { happy }\right)=5 / 10 & P\left(X_{t}=\operatorname{sing} \mid Z_{t}=\mathrm{sad}\right)=1 / 10 \\
P\left(X_{t}=\text { walk } \mid Z_{t}=\text { happy }\right)=3 / 10 & P\left(X_{t}=\text { walk } \mid Z_{t}=\mathrm{sad}\right)=2 / 10 \\
P\left(X_{t}=\mathrm{TV} \mid Z_{t}=\text { happy }\right)=2 / 10 & P\left(X_{t}=\mathrm{TV} \mid Z_{t}=\mathrm{sad}\right)=7 / 10
\end{array}
$$

Let's now calculate the joint probability of 
$$
\begin{array}{l}
P\left(X_{1: 2}=(\operatorname{sing}, \mathrm{TV}), Z_{1: 2}=(\text { happy, happy })\right) \\
P\left(X_{1: 2}=(\operatorname{sing}, \mathrm{TV}), Z_{1: 2}=(\text { happy }, \mathrm{sad})\right)
\end{array}
$$

The probability of z1, the first hidden state, being happy is 1/2. Given the z1 is happy, z2, the second hidden state, being happy is 4/5. Lastly, given z1 = happy, the probability of sing is 5/10. Given z2 = happy, the probability of TV is 2/10. This give us the following formula.
$$
P\left(X_{1: 2}=(\operatorname{sing}, T V), Z_{1: 2}=(\text {happy,happy})\right)=\frac{1}{2} *\left(\frac{4}{5}\right) *\left(\frac{5}{10} \cdot \frac{2}{10}\right)=\frac{1}{25}=0.04
$$

## Exercise: 
Calculate $P\left(X_{1: 2}=(\mathrm{sing}, \mathrm{TV}), Z_{1: 2}=(\text { happy }, \mathrm{sad})\right)$. Write the answer upto 3 decimal points ie. x.xxx to get full score.

In [7]:
#a = 0.000
a = 1/2 * 1/5 * 5/10 * 7/10
a = np.round(a,3)

In [8]:
grader.grade(test_case_id = 'test_case_HMM_happy_sad', answer = a)

Correct! You earned 3.0/3.0 points. You are a star!

Your submission has been successfully recorded in the gradebook.


**Markov Models and their steady state**

Let's look  at the properties of a Markov transition matrix, and in particular what it will converge to at steady state.  
Markov Matrices are square, but not symmetric, which means you need to be a little careful when computing eigenvectors (they have both left and right ones).

In [9]:
 # confirm that what the Markov sequence converges to
import numpy  as np
A = np.array([[0.8, 0.2], [0.6, 0.4]])
s = np.array([0, 1])
s1 = np.array([0.3, 0.7])  # starting point doesn't matter
print(s@A@A@A@A@A@A)
print(s1@A@A@A@A@A@A)
print([0.75, 0.25]@A)
print(A.T @np.array([0.75, 0.25]).T)
print('eigenvectors')
np.linalg.eig(A) #Hint to question below 

[0.749952 0.250048]
[0.7499712 0.2500288]
[0.75 0.25]
[0.75 0.25]
eigenvectors


(array([1. , 0.2]), array([[ 0.70710678, -0.31622777],
        [ 0.70710678,  0.9486833 ]]))

What is the largest eigenvalue of A above?

In [10]:
largest_eigenvalue = 1

In [11]:
grader.grade(test_case_id = 'test_case_largest_eigenvalue', answer = largest_eigenvalue)

Correct! You earned 1.0/1.0 points. You are a star!

Your submission has been successfully recorded in the gradebook.


In [12]:
# The transpose is closer to how we usually write things; but (again) 
# non-symmetric matrices don't give us the nice orthagonality we expect
print("reversed:",A.T@A.T@A.T@A.T@A.T@A.T@s.T)
# which of  the following are eigenvectors of A transpose?
print(A.T @ np.array([0.94868, 0.31622]).T)
print(A.T @ np.array([0.75, 0.25]).T)
print(A.T @ np.array([-0.707107, 0.707107]).T /0.2)


reversed: [0.749952 0.250048]
[0.948676 0.316224]
[0.75 0.25]
[-0.707107  0.707107]


In [13]:
# but if we're careful we can figure out  what repeated matrix multiplications
# will give us
(eig, eigv) = np.linalg.eig(A.T)
print('eigenvalues', eig)
print('left eigenvectors', eigv)
#A.T @A.T @A.T @A.T @A.T @A.T @ s.T
first_eigv = eigv[:,0]
print('Eigenvectors are normalized using L2 norm, but we want to find a probability,')
print('which is L1-normalized')
print('first eigenvector, rescaled:', first_eigv/np.sum(first_eigv))


eigenvalues [1.  0.2]
left eigenvectors [[ 0.9486833  -0.70710678]
 [ 0.31622777  0.70710678]]
Eigenvectors are normalized using L2 norm, but we want to find a probability,
which is L1-normalized
first eigenvector, rescaled: [0.75 0.25]



What is the main assumption regarding the probability of different states across time in a Hidden Markov Model? (There may be more than one answer, write the answer in a list of two elements eg [1,2], If you think only a single answer is correct write the answer for eg like [1,0])
1. Observations x_t at time t are conditionally independent of all other variables given z_t
2. Observation x_t at time t depends on the observation at time t-1 i.e. x_{t-1}
3. Observation x_t at time t depends only on the current state i.e. z_t
4. Observation x_t at time t is explicitly dependent on all past states i.e. z_0 to z_t

In [14]:
ans = [1,3]

In [15]:
grader.grade(test_case_id = 'test_case_mcq', answer = ans)

Correct! You earned 2.0/2.0 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## Submitting to the Autograder

First of all, please run your notebook from beginning to end and ensure you are getting all the points from the autograder!

Now go to the File menu and choose "Download .ipynb".  Go to [Gradescope](https://www.gradescope.com/courses/409970) and:

1. From "File" --> Download both .ipynb and .py files
1. Name these files `HMM_WS.ipynb` and `HMM_WS.py` respectively
1. Sign in using your Penn email address (if you are a SEAS student we recommend using the Google login) and ensure  your class is "CIS 5200"
1. Select **Worksheet: HMM**
1. Upload both files
1. PLEASE CHECK THE AUTOGRADER OUTPUT TO ENSURE YOUR SUBMISSION IS PROCESSED CORRECTLY!

You should be set! Note that this assignment has 10 autograded points that will show up upon submission. Points are awarded based on a combination of correctness and sufficient effort. 